In [1]:
import os
from PIL import Image as img
import matplotlib.pyplot as plt
import numpy as np
import cv2
import tensorflow as tf
from keras import layers
from keras.models import Model
from Network import *
from preprocess import *
import time
from IPython import display

In [2]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [2]:
BUFFER_SIZE = 60000
BATCH_SIZE = 3
CRITIC_ITERATION = 5
WEIGHT_CLIP = 0.01


In [3]:
generatore = generator()
discriminator = discrim()
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy()

In [4]:
def discriminator_loss(real_output, fake_output):
    a = np.array(real_output)
    c = np.array(fake_output)
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    b = np.array(real_loss)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss   
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-5)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generatore,
                                 discriminator=discriminator)

In [5]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
def train_step(images,noise):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generatore(noise, training=True)
      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)
      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)      
    gradients_of_generator = gen_tape.gradient(gen_loss, generatore.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generatore.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


In [6]:
def train_discrim_step(images,image_noise):
    with tf.GradientTape() as disc_tape:      
      generated_images = generatore(image_noise)
      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)
      disc_loss = discriminator_loss(real_output, fake_output)      
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
def train_gen_step(image_noise):
    with tf.GradientTape() as gen_tape:
        generated_images = generatore(image_noise)  
        gen_fake = discriminator(generated_images)
        gen_loss = generator_loss(gen_fake)
    gradients_of_generator = gen_tape.gradient(gen_loss, generatore.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generatore.trainable_variables))


In [7]:
def generate_and_save_images(model, epoch, test_input):
  cv2.imwrite(f"Image/image{epoch}_With_Noise.png",test_input[0]*127.5+127.5)
  predictions = np.array(model(test_input, training=False))  
  cv2.imwrite(f"Image/image{epoch}_Without_Noise.png",predictions[0]*127.5+127.5)
  plt.show()

In [8]:
image_batch = generate_random_value(2)
seed = generate_random_noise_value(image_batch.copy(),2)

In [9]:
def train(nb_batch,epochs):
  for epoch in range(epochs):
    start = time.time()

    for _ in range(nb_batch):
      image_batch = generate_random_value(BATCH_SIZE)
      image_noise = generate_random_noise_value(image_batch,BATCH_SIZE)
      image_batch = resize_value(image_batch,BATCH_SIZE)
      train_step(image_batch,image_noise)

    # Produce images for the GIF as you go
    display.clear_output(wait=True)
    generate_and_save_images(generatore,
                             epoch + 1,
                             seed.copy())

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generatore,
                           epochs,
                           seed.copy())

In [10]:
train(1,4)

Time for epoch 4 is 51.084736347198486 sec


In [ ]:
image_batch = generate_random_value(2)
seed = generate_random_noise_value(image_batch.copy(),2)
generate_and_save_images(generatore,
                             0,
                             seed.copy())